# Unofficial ACI Guide

## Python 3 Healthcheck Example

This is a simple Python example demonstrating how to obtain Healthcheck information.

In [1]:
import requests
import json
import pandas as pd

We'll read the auth info from config.py. 
Just edit that file to include your controller IP/hostname, username, and password.

In [2]:
from config import controller, username, password

In [3]:
# Set some variables to construct our login URLs

base_url = "https://" + str(controller) + "/api/"
auth_bit = "aaaLogin.json"

auth_url = base_url + auth_bit
auth_url

'https://10.18.188.101/api/aaaLogin.json'

In [4]:
# JSON auth data 
auth_data = {
  "aaaUser":{
    "attributes":{
      "name":username,
      "pwd":password
    }
  }
}

## Construct the Request

In [5]:
# Uncomment this to suppress certificate warnings.
requests.packages.urllib3.disable_warnings() 

In [6]:
# We use verify=False to allow self-signed certificates. 
# Change to "verify=True" or just remove verify=False if you use an internal CA or a valid chain cert.
s = requests.session()
s.post(auth_url, json=auth_data, verify=False)

<Response [200]>

Fault Summary:
https://10.18.188.101/api/node/class/faultSummary.json?query-target-filter=and(not(wcard(polUni.dn, "__ui_")),and())&order-by=faultSummary.severity|desc&page=0&page-size=100```
 
Health Score: 
https://10.18.188.101/api/node/mo/topology/HDfabricOverallHealth5min-0.json



In [7]:
#https://apic-ip-address/api/node/mo/topology/HDfabricOverallHealth5min-0.json
# What's the URL?
health = "node/mo/topology/HDfabricOverallHealth5min-0.json"
info = "node/mo/info.json"
fault = "node/class/faultSummary.json"
health_url = base_url + health
info_url = base_url + info
fault_url = base_url + fault

In [33]:
s_response = s.get(health_url, verify=False)
s_health_data = s_response.json()

In [34]:
s_health_data

{'totalCount': '1',
 'imdata': [{'fabricOverallHealthHist5min': {'attributes': {'childAction': '',
     'cnt': '30',
     'dn': 'topology/HDfabricOverallHealth5min-0',
     'healthAvg': '95',
     'healthMax': '95',
     'healthMin': '95',
     'healthSpct': '0',
     'healthThr': '',
     'healthTr': '0',
     'index': '0',
     'lastCollOffset': '300',
     'repIntvEnd': '2019-05-02T14:14:02.142-05:00',
     'repIntvStart': '2019-05-02T14:09:02.037-05:00',
     'status': ''}}}]}

In [37]:
health_score = s_health_data['imdata'][0]
health_score

{'fabricOverallHealthHist5min': {'attributes': {'childAction': '',
   'cnt': '30',
   'dn': 'topology/HDfabricOverallHealth5min-0',
   'healthAvg': '95',
   'healthMax': '95',
   'healthMin': '95',
   'healthSpct': '0',
   'healthThr': '',
   'healthTr': '0',
   'index': '0',
   'lastCollOffset': '300',
   'repIntvEnd': '2019-05-02T14:14:02.142-05:00',
   'repIntvStart': '2019-05-02T14:09:02.037-05:00',
   'status': ''}}}

In [42]:
health_score['fabricOverallHealthHist5min']['attributes']
five_min_avg = health_score['fabricOverallHealthHist5min']['attributes']['healthAvg']
five_min_max = health_score['fabricOverallHealthHist5min']['attributes']['healthMax']
five_min_min = health_score['fabricOverallHealthHist5min']['attributes']['healthMin']

In [29]:
s_response = s.get(fault_url, verify=False)
s_fault = s_response.json()

In [9]:
# Let's print out formatted JSON response for easy viewing
#print(json.dumps(s_ep, indent=4, sort_keys=True))

In [12]:
#object_list = s_fault['imdata']

In [13]:
#object_list[0]

In [14]:
#dn = object_list[0]['faultSummary']['attributes']['dn']
#descr = object_list[0]['faultSummary']['attributes']['descr']
#severity = object_list[0]['faultSummary']['attributes']['severity']
#code = object_list[0]['faultSummary']['attributes']['code']
#type = object_list[0]['faultSummary']['attributes']['type']

In [15]:
# Set up some empty lists for tenant, endpoint group, endpoint, IP address, MAC address, and encapsulation. 
descr_list = []
severity_list = []
code_list = []
type_list = []
dn_list = []
cause_list = []

# The imdata dictionary key has a single value which is a list of objects
# that we'll extract into "object_list" for easier ereferenece... 
object_list = s_fault['imdata']

# ...and iterate over it to retrieve the dictionary values we want.
# This is a bit simpler than accessing each element directly like we did above. 
# We're printing the object on each iteration for convenience.

for object in object_list:
    #print(object)
    dn_list.append(object['faultSummary']['attributes']['dn'])
    descr_list.append(object['faultSummary']['attributes']['descr'])
    severity_list.append(object['faultSummary']['attributes']['severity'])
    code_list.append(object['faultSummary']['attributes']['code'])
    type_list.append(object['faultSummary']['attributes']['type'])
    cause_list.append(object['faultSummary']['attributes']['cause'])

In [16]:
# Let's take our list and zip them up.
list_of_objects = zip(code_list,severity_list,descr_list,cause_list,type_list,dn_list)

In [17]:
# Convert to list of tuples and stuff it into a Pandas dataframe...
df_input = list(list_of_objects)
df = pd.DataFrame(df_input, columns=("Code","Severity","Description","Cause","Type","DN"))
# Note, if you change the column order in the dataframe, be sure to change order of 
# lists zipped into list_of_ep above.

In [18]:
# Print the dataframe in Jupyter.
df

# Use this to print in a standalone script.
#print(df)

,Code,Severity,Description,Cause,Type,DN
0,F0103,major,This fault occurs when a physical interface on...,port-down,operational,fltcode-F0103
1,F2543,major,This fault is raised when there are DHCP issues,invalid-configuration,operational,fltcode-F2543
2,F0756,minor,This fault occurs when a configured target of ...,resolution-failed,config,fltcode-F0756
3,F2650,minor,Plugin has failed to start.,failed-to-start,operational,fltcode-F2650
4,F0053,minor,This fault occurs when a configuration export/...,configuration-failed,config,fltcode-F0053
5,F1322,warning,This fault occurs when a fan fails,equipment-fan-failed,operational,fltcode-F1322
6,F1040,warning,The object refers to an object that was not fo...,resolution-failed,config,fltcode-F1040
7,F3057,warning,This fault is raised when APIC Controller prod...,product-not-registered,config,fltcode-F3057
8,F3062,major,This fault is raised when product license eval...,evaluation-period-expired,config,fltcode-F3062
9,F2740,warning,This fault occurs when port speed is configure...,invalid-port-speed-configured,communications,fltcode-F2740


In [28]:
# Let's extract just the tenants and endpoints, create a groupby object, 
# then count the number of endpoints per tenant.
#new_df = df[['Severity','Type']]
summary_df = df.groupby(['Severity', 'Type']).agg({'Cause':'count'})
summary_df

# Use this to print in a standalone script.
#print(summary_df)

Cause
Severity Type                 
major    communications      1
         config              1
         operational         3
minor    config              4
         environmental       1
         operational         1
warning  communications      2
         config              8
         operational         2